In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -q /content/drive/My\ Drive/upper_lower_zip.zip

In [0]:
!unzip -q /content/testing_data_zip_2.zip

In [0]:
import torch
torch.cuda.current_device()

0

In [0]:
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import transforms
import torchvision.models as models
import numpy as np
from tqdm import tqdm
from PIL import Image
from PIL import ImageFile
use_cuda = torch.cuda.is_available()

In [0]:
img_std = [0.229,0.224,0.225]
img_mean = [0.485,0.456,0.406]
batch_size = 32

train_transforms = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(img_mean,img_std)
                                     ])
train_dir = "/content/upper_lower/valid/train"
valid_dir = "/content/upper_lower/valid/val"
test_dir = "/content/upper_lower/test"

#train_dataset = datasets.ImageFolder(train_dir,transform=train_transforms)
#valid_dataset = datasets.ImageFolder(valid_dir,transform=train_transforms)
test_dataset = datasets.ImageFolder(test_dir,transform=train_transforms)

#trainloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
#validloader = torch.utils.data.DataLoader(valid_dataset,batch_size=batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

loaders = {
    #'train':trainloader,
    #'valid':validloader,
    'test':testloader
}

In [0]:
class FashionNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder_net = models.resnet50(pretrained=True)
    self.encoder_net = nn.Sequential(*list(self.encoder_net.children())[:-1])
    
    
    
    self.fc1 = nn.Sequential(nn.Dropout(0.4),
                            nn.Linear(4096,512),
                            nn.BatchNorm1d(num_features=512),
                            nn.ReLU(True),
                            nn.Dropout(0.2),
                            nn.Linear(512,2),
                            nn.LogSoftmax(dim=1))
    
  def forward(self,x):
    upper_im = x[:,:,:,:224]
    lower_im = x[:,:,:,224:]
    upper_enc = self.encoder_net(upper_im).view((-1,2048))
    lower_enc = self.encoder_net(lower_im).view((-1,2048))

    score = self.fc1(torch.cat((upper_enc,lower_enc),dim=1))

    return score
    
model = FashionNet()
if use_cuda:
  model.cuda()
model.load_state_dict(torch.load("/content/drive/My Drive/upper_lower_4.pt"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
import torch.optim as optim

criterion = nn.NLLLoss()

optimizer = optim.Adam(model.parameters(),lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.2)

In [0]:
def train(n_epochs,model,criterion,optimizer,scheduler,loaders,use_cuda,save_path):
  valid_loss_min = np.Inf
  valid_acc_min = 0
  
  for epoch in range(1,n_epochs+1):
    valid_loss = 0
    train_loss = 0
    valid_corrects = 0
    train_corrects = 0
    
    scheduler.step()
    
    # Training loop
    model.train()
    for data,target in tqdm(loaders['train']):
      if use_cuda:
        data,target = data.cuda(),target.cuda()
        
      optimizer.zero_grad()
      
      output = model(data)
      
      loss = criterion(output,target)
      
      loss.backward()
      
      optimizer.step()
      
      train_loss += loss.item()
      
      pred = output.data.max(1,keepdim=True)[1]
      train_corrects += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
      
    
    #Validation loop
    model.eval()
    for data,target in tqdm(loaders['valid']):
      if use_cuda:
        data,target = data.cuda(),target.cuda()
       
      with torch.no_grad():
        output = model(data)
      
      loss = criterion(output,target)
      
      valid_loss += loss.item()
      
      pred = output.data.max(1,keepdim=True)[1]
      valid_corrects += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
    
    valid_loss = valid_loss / len(loaders['valid'])
    train_loss = train_loss / len(loaders['train'])
    valid_accuracy = valid_corrects / (len(loaders['valid'])*batch_size)
    train_accuracy = train_corrects / (len(loaders['train'])*batch_size)
      
    #Print epoch details
    print("Epoch: {}, Training loss: {}, Training accuracy: {}, Valid loss: {}, Valid accuracy; {}".format(epoch,train_loss,train_accuracy,valid_loss,valid_accuracy))
    
    if valid_loss < valid_loss_min:
      valid_loss_min = valid_loss
      valid_acc_min = valid_accuracy
      checkpoint = model.state_dict()
      torch.save(checkpoint,save_path)
      
  print("The best model had a validation loss of {} and an accuracy of {}".format(valid_loss_min,valid_acc_min))

In [0]:
model = train(15,model,criterion,optimizer,scheduler,loaders,use_cuda,"/content/drive/My Drive/upper_lower_4.pt")

In [0]:
def test(loaders, model, criterion, use_cuda):

    # Monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    model.eval()
    for batch_idx, (data, target) in tqdm(enumerate(loaders['test'])):
        # Move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        
        # Find the model's prediction
        output = model(data)
        
        # Calculate the loss
        loss = criterion(output, target)
        
        # Update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        
        # Update the test accuracy
        pred = output.data.max(1, keepdim=True)[1]
        
        for idx in range(len(target)):
          if target[idx] == 1:
            if pred[idx] == 1:
              TP += 1
            else:
              FN += 1
          else:
            if pred[idx] == 0:
              TN += 1
            else:
              FP += 1
    accuracy = (TP+TN)/(TP+FP+FN+TN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    f1 = 2*(recall * precision) / (recall + precision)
     
    # Print test loss and accuracy
    print('accuracy: {}, precision: {}, recall: {}, f1: {}'.format(accuracy,precision,recall,f1))

In [0]:
test(loaders,model,criterion,use_cuda)

666it [03:24,  3.22it/s]

accuracy: 0.6570476726726727, precision: 0.682796286182414, recall: 0.5866178678678678, f1: 0.6310635505527232
